In [1]:
import pandas as pd
from pandas import DataFrame, Series
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [3]:
df = pd.read_csv('Matt.csv')
df.head()

,Player,FantPos,Age,G,GS,Cmp,Att,Yds,TD,Int,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,PosRank,N_PPR_OvRank,PPR_OvRank,Year
0,Derrick Henry,RB,26,16,16,0,0,0,0,0,...,1,0,314,333.1,341.1,323.6,1,11.0,12.0,2020
1,Alvin Kamara,RB,25,15,10,0,0,0,0,0,...,0,0,295,377.8,383.8,336.3,2,13.0,4.0,2020
2,Dalvin Cook,RB,25,14,14,0,0,0,0,0,...,3,0,294,337.8,346.8,315.8,3,14.0,11.0,2020
3,Davante Adams,WR,28,14,14,0,0,0,0,0,...,0,0,243,358.4,362.4,300.9,1,21.0,8.0,2020
4,Travis Kelce,TE,31,15,15,1,2,4,0,0,...,1,0,208,312.8,316.8,260.3,1,30.0,17.0,2020


In [27]:
df.dtypes

Player           object
FantPos          object
Age               int64
G                 int64
GS                int64
Cmp               int64
Att               int64
Yds               int64
TD                int64
Int               int64
Att.1             int64
Yds.1             int64
Y/A             float64
TD.1              int64
Tgt               int64
Rec               int64
Yds.2             int64
Y/R             float64
TD.2              int64
Fmb               int64
FL                int64
TD.3              int64
2PM               int64
2PP               int64
FantPt            int64
PPR             float64
DKPt            float64
FDPt            float64
PosRank           int64
N_PPR_OvRank    float64
PPR_OvRank      float64
Year              int64
dtype: object

In [31]:
#crete variables
xvars =['GS', 'Cmp', 'Yds', 'TD', 'Att', 'Int', 'Att.1', 'Yds.1', 'Y/A', 'TD.1','Tgt', 'Rec', 'Yds.2', 'Y/R','TD.2', 'Fmb', 'FL', 'TD.3']
yvar = 'FantPos'

In [32]:
#create train/ test split
train, test = train_test_split(df, test_size=0.20)

In [33]:
model = RandomForestClassifier(n_estimators=100)
model.fit(train[xvars], train[yvar])

RandomForestClassifier()

In [35]:
test['pos_hat'] = model.predict(test[xvars])
test['correct'] = (test['pos_hat'] == test['FantPos'])
test['correct'].mean()

0.8245614035087719

In [36]:
model.predict_proba(test[xvars])

array([[0.08      , 0.22      , 0.17      , 0.53      ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.55      , 0.45      ],
       [0.01      , 0.        , 0.22      , 0.77      ],
       [0.        , 0.87      , 0.        , 0.13      ],
       [0.        , 0.        , 0.13      , 0.87      ],
       [0.        , 0.96      , 0.        , 0.04      ],
       [0.        , 0.        , 0.48      , 0.52      ],
       [0.        , 0.        , 0.01      , 0.99      ],
       [0.        , 0.        , 0.11      , 0.89      ],
       [0.        , 0.02      , 0.81      , 0.17      ],
       [0.        , 0.88      , 0.        , 0.12      ],
       [0.        , 0.99      , 0.        , 0.01      ],
       [0.        , 0.96      , 0.        , 0.04      ],
       [0.        , 0.        , 0.18      , 0.82      ],
       [0.        , 0.        , 0.25      , 0.75      ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.01      ,

In [37]:

probs = DataFrame(model.predict_proba(test[xvars]),
                  index=test.index,
                  columns=model.classes_)
probs.head()

,QB,RB,TE,WR
176,0.08,0.22,0.17,0.53
166,0.00,0.00,0.00,1.00
229,0.00,0.00,0.55,0.45
227,0.01,0.00,0.22,0.77
255,0.00,0.87,0.00,0.13


In [40]:
results = pd.concat([
    test[['Player', 'FantPos', 'pos_hat', 'correct']],
    probs], axis=1)
results.head()

,Player,FantPos,pos_hat,correct,QB,RB,TE,WR
176,Isaiah McKenzie,WR,WR,True,0.08,0.22,0.17,0.53
166,George Kittle,TE,WR,False,0.00,0.00,0.00,1.00
229,Miles Boykin,WR,TE,False,0.00,0.00,0.55,0.45
227,Golden Tate,WR,WR,True,0.01,0.00,0.22,0.77
255,La'Mical Perine,RB,RB,True,0.00,0.87,0.00,0.13


In [42]:
results.groupby('FantPos')[['correct', 'QB', 'RB', 'WR', 'TE']].mean()

,correct,QB,RB,WR,TE
FantPos,,,,,
QB,0.909091,0.909091,0.080000,0.007273,0.003636
RB,0.911765,0.005882,0.834877,0.098235,0.061005
TE,0.608696,0.000870,0.028087,0.452609,0.518435
WR,0.847826,0.023043,0.072094,0.676304,0.228558


In [43]:
# feature importance
Series(model.feature_importances_, xvars).sort_values(ascending=False)

Att.1    0.130412
Y/R      0.126362
Yds.2    0.108090
Tgt      0.082430
Yds.1    0.081421
Att      0.071500
Rec      0.068244
Y/A      0.067822
GS       0.060962
Cmp      0.053903
Yds      0.038845
TD.1     0.027606
TD.3     0.022975
TD.2     0.020133
Int      0.012246
Fmb      0.011803
FL       0.007931
TD       0.007316
dtype: float64